# Single Heart Rate Monitor Example

This is a minimal demonstration of how to set up and use Bleak-FSM.

This examples uses Bluetooth Low Energy (BLE) Heart Rate service.
+ It must support Bluetooth, not just ANT+
+ The GATT Characteristic required is: `00002a37-0000-1000-8000-00805f9b34fb`
+ You may use the [`bluetooth_dissect.py` tool](https://github.com/tensorturtle/bluetooth_dissect) to check if your device supports that GATT Characteristic.

Example compatible devices:
+ Magene H603
+ Wahoo TICKR & TICKR Fit
+ WHOOP 4.0 (Broadcast Heart Rate ON)
+ and many more advertised as "supports Bluetooth Heart Rate"

If you don't have access to a BLE Heart Rate device or want to a different BLE GATT characteristic, see the [Migration Guide](migration_guide.ipynb)


## How to run this Notebook

+ Make sure your computer's bluetooth adapter is powered on.
+ At first, I recommend stepping through each cell, reading the code and explanations.
+ You will need to set your Heart Rate monitor's address the first time.
+ At any time, `print(model.state)` to confirm which state you're in. This will help you figure out what you can do from there.

In [1]:
!pip3 install -r -q requirements.txt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '-q'
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


# Setup

In [2]:
import asyncio

In [3]:
from bleak_fsm import machine, BleakModel

In [4]:
model = BleakModel()
machine.add_model(model)

Toggle this `USE_PYCYCLING` flag to see how this library can accomodate using standard, raw BLeakClient or Pycycling-wrapped BleakClient.

In [5]:
USE_PYCYCLING = True

In [6]:
# Define callbacks

if USE_PYCYCLING:
    from pycycling.heart_rate_service import HeartRateService
    model.wrap = lambda client: HeartRateService(client)
    model.enable_notifications = lambda client: client.enable_hr_measurement_notifications()
    model.disable_notifications = lambda client: client.disable_hr_measurement_notifications()
    def handle_hr_measurement(value):
        print("Using Pycycling wrapper around BleakClient")
        print(f"Heart Rate: {value}")
    model.set_measurement_handler = lambda client: client.set_hr_measurement_handler(handle_hr_measurement)
else:
    HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID = "00002a37-0000-1000-8000-00805f9b34fb"
    model.enable_notifications = lambda client: client.start_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID, handle_hr_measurement)
    model.disable_notifications = lambda client: client.stop_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID)
    def handle_hr_measurement(sender, data):
        print("Using raw BleakClient")
        print(f"Data received from {sender}: {data}")
        flag = data[0]
        heart_rate = data[1]
        print(f"Heart Rate: {heart_rate} beats per minute")
    model.set_measurement_handler = lambda client: handle_hr_measurement


# Usage

Since we're in a Jupyter notebook, we need to use `create_task()` instead of `get_event_loop().run_until_complete()` to use the existing event loop which Jupyter runs on.

The `BleakModel` class represents this system's bluetooth adapter. This library currently supports one bluetooth adapter. We call the `start_scan()` class method.

In [7]:
await BleakModel.start_scan()

True

In [8]:
async def print_as_we_go(max_devices=30):
    for i in range(40):
        if len(BleakModel.bt_devices) >= max_devices:
            print(f"{len(BleakModel.bt_devices)} devices found. Stopping scan.")
            await model.stop_scan()
            break
        print(f"{len(BleakModel.bt_devices)} devices found in {round(i*0.05,2)} seconds.")
        await asyncio.sleep(0.05)

await print_as_we_go()

0 devices found in 0.0 seconds.
1 devices found in 0.05 seconds.
1 devices found in 0.1 seconds.
2 devices found in 0.15 seconds.
3 devices found in 0.2 seconds.
3 devices found in 0.25 seconds.
3 devices found in 0.3 seconds.
4 devices found in 0.35 seconds.
4 devices found in 0.4 seconds.
4 devices found in 0.45 seconds.
4 devices found in 0.5 seconds.
5 devices found in 0.55 seconds.
5 devices found in 0.6 seconds.
5 devices found in 0.65 seconds.
5 devices found in 0.7 seconds.
6 devices found in 0.75 seconds.
6 devices found in 0.8 seconds.
6 devices found in 0.85 seconds.
6 devices found in 0.9 seconds.
6 devices found in 0.95 seconds.
6 devices found in 1.0 seconds.
6 devices found in 1.05 seconds.
6 devices found in 1.1 seconds.
6 devices found in 1.15 seconds.
6 devices found in 1.2 seconds.
6 devices found in 1.25 seconds.
6 devices found in 1.3 seconds.
6 devices found in 1.35 seconds.
6 devices found in 1.4 seconds.
6 devices found in 1.45 seconds.
6 devices found in 1.5 se

In [9]:
await BleakModel.stop_scan()

True

In [10]:
BleakModel.bt_devices

{'14863C4D-BF71-4EA3-C6B4-98001056AAF8': (BLEDevice(14863C4D-BF71-4EA3-C6B4-98001056AAF8, WHOOPDEDOO),
  AdvertisementData(local_name='WHOOPDEDOO', service_uuids=['0000180d-0000-1000-8000-00805f9b34fb'], rssi=-53)),
 '3FEC3389-7F84-AFC4-6EF1-BDE6CD37F862': (BLEDevice(3FEC3389-7F84-AFC4-6EF1-BDE6CD37F862, Green Apple),
  AdvertisementData(manufacturer_data={76: b'\x10\x07r\x1f7;M\xdaH'}, tx_power=7, rssi=-43)),
 '9E41F52B-BDEF-3C85-F51D-E0DE23B7CB41': (BLEDevice(9E41F52B-BDEF-3C85-F51D-E0DE23B7CB41, Study),
  AdvertisementData(manufacturer_data={76: b'\x0f\x05\x90\x00\xa5=\x14\x10\x02-\x04'}, tx_power=6, rssi=-43)),
 'C01DDA98-E201-495D-0D2D-AF97315B1112': (BLEDevice(C01DDA98-E201-495D-0D2D-AF97315B1112, None),
  AdvertisementData(manufacturer_data={76: b'\t\x08\x13J\xac\x1e\x01Q\x1bX\x13\x08\n\x18\x98F\x8d\x9a\xdd\x00'}, rssi=-45)),
 '37DA5EA4-DB46-2D6D-9E8F-A7B7D4DBCCA4': (BLEDevice(37DA5EA4-DB46-2D6D-9E8F-A7B7D4DBCCA4, GiGA Genie 3_F70E),
  AdvertisementData(local_name='GiGA Genie 3_

Copy-and-paste the key corresponding to the device that you want to connect to:

In [11]:
target_address = "14863C4D-BF71-4EA3-C6B4-98001056AAF8"

Or search for the name of the device from the values and retrieve its key.

While most consumer heart rate monitors have a name, this name might be not unique or missing.

In [12]:
target_name = "WHOOPDEDOO"

target_address = ""
for address, (ble_device, advertisement_data) in BleakModel.bt_devices.items():
    if ble_device.name == target_name:
        target_address = address
        print(f"Found {target_name} at {target_address}")

Found WHOOPDEDOO at 14863C4D-BF71-4EA3-C6B4-98001056AAF8


Anyhow, we will attempt to connect to the following address:

In [13]:
print(target_address)

14863C4D-BF71-4EA3-C6B4-98001056AAF8


In [14]:
await model.set_target(target_address)

True

 When you provide a target address to the `model`, its state changes to `TargetSet`

In [15]:
model.state

'TargetSet'

From `TargetSet` state, you may now attempt to connect to the device.

By default, if it has been over 120 seconds since the last scan, if `auto_rescan=True`, the model will automatically re-scan and then attempt the connection.
These parameters may be changed in the `BleakModule` instantiation above.

In [16]:
success = await model.connect(timeout=0.01)

if not success:
    print("Correctly failed to connect in time")

Correctly failed to connect in time


In [17]:
success = await model.connect(timeout=5)
if success:
    print("Connected thanks to long timeout")
else:
    raise Exception("Failed to connect. Something else is up.")

Connected thanks to long timeout


Sidebar: Note how the above cell returns `True`. All transition methods on `model` returns True if the transition was successful. Therefore you may explicitly define happy-path and exception-path in different ways.

For debugging,
```python
assert await model.connect(), "Model failed to connect"
```

Or, more properly check the return value
```python
success = await model.connect()
if not succcess:
    await model.clean_up() # this catch-all method brings the model back to "Init"
    raise Exception("Failed to connect")
```

Or, we can check the `.state` afterwards, like so:

In [18]:
if model.state != "Connected":
    raise Exception("Failed to connect. If the device was not properly disconnected last time, restart either the device or bluetooth adapter")

In order to receive/send actual data from/to the connected device, we need to start a stream.

This will run forever until `model.disconnect()` is called, so for now let's sleep for several seconds and then call that.

In [19]:
await model.stream()

True

In [20]:
await asyncio.sleep(5)

Using Pycycling wrapper around BleakClient
Heart Rate: HeartRateMeasurement(sensor_contact=False, bpm=70, rr_interval=[], energy_expended=None)
Using Pycycling wrapper around BleakClient
Heart Rate: HeartRateMeasurement(sensor_contact=False, bpm=70, rr_interval=[], energy_expended=None)
Using Pycycling wrapper around BleakClient
Heart Rate: HeartRateMeasurement(sensor_contact=False, bpm=70, rr_interval=[], energy_expended=None)
Using Pycycling wrapper around BleakClient
Heart Rate: HeartRateMeasurement(sensor_contact=False, bpm=70, rr_interval=[], energy_expended=None)


Bleak-FSM encapsulates a kind of bug in Bleak, where it is not possible to re-use a connected client. 

That is, we shouldn't step back to "Connected" state from "Streaming", because we can't go from "Connected" back to "Streaming".

Instead we need to go back two steps to "TargetSet", re-establish a connection, and then we can stream again.

Bleak-FSM enforces this behavior for your own safety.
There exists not method for you to even try to go from "Streaming" to "Connected".


In [21]:
await model.disconnect()

True

Whenever the application terminates, or otherwise no longer needs to communicate with the bluetooth device, it should always call `disconnect()` on the model. The model state should always e either "TargetSet" or "Init".

In [22]:
model.state

'TargetSet'

In [23]:
await model.unset_target()

True

In [24]:
model.state

'Init'

For convenience, you may call `await model.clean_up()` from any state to come back to "Init"

In [25]:
await model.clean_up()

True

At this point, we have returned to 'Init' state, and you may 'start_scan', 'set_target_address', 'connect', etc.

# Again, with Errors

Let's review the above tutorial, this time also checking that the proper errors are being emitted when you  attempt illegal things.

We'll discuss how best to handle those errors.

Let's try to skip a step in the Finite State Machine.

From `Init`, we are only allowed to go to `TargetSet`.

If we try to go to `Connect`, we should get a `MachineError`:

In [26]:
model.state

'Init'

In [27]:
from transitions import MachineError
try:
    await model.connect()
except MachineError as e:
    print("Correct error raised:")
    print(e)

ERROR:transitions.extensions.asyncio:Exception was raised while processing the trigger: "Can't trigger event connect from state Init!"


Correct error raised:
"Can't trigger event connect from state Init!"


In order to go to `TargetSet`, we need an address.

To get an address, we need to scan. For simplicity, we'll just `await` it instead of sending it to the background as we did above.

In [28]:
await model.start_scan()

True

This puts us in "Scaning" state:

In [29]:
print(model.state)

Init


From "Scanning", transitioning to any other state besides "Init" is illegal:

In [30]:
try:
    await model.connect()
except MachineError as e:
    print("Correct error raised:")
    print(e)

ERROR:transitions.extensions.asyncio:Exception was raised while processing the trigger: "Can't trigger event connect from state Init!"


Correct error raised:
"Can't trigger event connect from state Init!"


Let's stop the scan.

In [31]:
await asyncio.sleep(3)
await model.stop_scan()

True

In [32]:
print(model.state)

Init


As before, when scanning is done, the results are saved in the class variable `BleakModel.bt_devices`.

Let's throw a wrench into the gears and use an invalid address.
We expect a `BleakFSMError`

In [33]:
from bleak_fsm import BleakFSMError

In [34]:
incorrect_target_address = "some_incorrect_nonexistent_address"
try:
    await model.set_target(incorrect_target_address)
except BleakFSMError as e:
    print("Correct error raised:")
    print(e)

ERROR:transitions.extensions.asyncio:Exception was raised while processing the trigger: Address some_incorrect_nonexistent_address not found in discovered devices


Correct error raised:
Address some_incorrect_nonexistent_address not found in discovered devices


Now back on track, let's set the target with a correct target address determined above:

In [35]:
await model.set_target(target_address)

True

We will now test what happens if we try to connect when the last scan is too old. 

Notice how we instantiated `model` as:

```python
model = BleakModel(
    scan_stale_time=3,
    auto_rescan=False,
)
```

If we sleep 5 seconds, then the scanned results will be considered stale. And since `auto_rescan` was set to False, we will get a `StaleScanError`:

In [36]:
from bleak_fsm import StaleScanError

In [37]:
await asyncio.sleep(5) # sleeping in order to trigger stale scan error

try:
    await model.connect()
except StaleScanError as e:
    print("Correct error raised:")
    print(e)


In [38]:
await model.clean_up() # always remember to return the device to "Init" state before exiting the program

True